# linux 1.0 notes

## net/inet/skbuff

```c
struct sk_buff {
  unsigned long			magic_debug_cookie;
  struct sk_buff		*volatile next;
  struct sk_buff		*volatile prev;
  struct sk_buff		*volatile link3;
  struct sk_buff		*volatile* list;
  struct sock			*sk;
  volatile unsigned long	when;	/* used to compute rtt's	*/
  struct device			*dev;
  void				*mem_addr;
  union {
	struct tcphdr	*th;
	struct ethhdr	*eth;
	struct iphdr	*iph;
	struct udphdr	*uh;
	struct arphdr	*arp;
	unsigned char	*raw;
	unsigned long	seq;
#ifdef CONFIG_IPX	
	ipx_packet	*ipx;
#endif	
  } h;
  struct iphdr		*ip_hdr;		/* For IPPROTO_RAW */
  unsigned long			mem_len;
  unsigned long 		len;
  unsigned long			fraglen;
  struct sk_buff		*fraglist;	/* Fragment list */
  unsigned long			truesize;
  unsigned long 		saddr;
  unsigned long 		daddr;
  int				magic;
  volatile char 		acked,
				used,
				free,
				arp;
  unsigned char			tries,lock;	/* Lock is now unused */
  unsigned short		users;		/* User count - see datagram.c (and soon seqpacket.c/stream.c) */
  unsigned long			padding[0];
  unsigned char			data[0];
};
```

* last field are data buffer. So this struct has a dynamic size. 

* 
![skbuff01.png](imgs/skbuff01.png)

------------

```c
/*
 *	Get a clone of an sk_buff. This is the safe way to peek at
 *	a socket queue without accidents. Its a bit long but most
 *	of it acutally ends up as tiny bits of inline assembler
 *	anyway. Only the memcpy of upto 4K with ints off is not
 *	as nice as I'd like.
 */

struct sk_buff *skb_peek_copy(struct sk_buff *volatile* list)
{
	struct sk_buff *orig,*newsk;
	unsigned long flags;
	unsigned int len;
	/* Now for some games to avoid races */

	do
	{
		save_flags(flags);
		cli();
		orig=skb_peek(list);
		if(orig==NULL)
		{
			restore_flags(flags);
			return NULL;
		}
		IS_SKB(orig);
		len=orig->truesize;
		restore_flags(flags);

		newsk=alloc_skb(len,GFP_KERNEL);	/* May sleep */

		if(newsk==NULL)		/* Oh dear... not to worry */
			return NULL;

		save_flags(flags);
		cli();
		if(skb_peek(list)!=orig)	/* List changed go around another time */
		{
			restore_flags(flags);
			newsk->sk=NULL;
			newsk->free=1;
			newsk->mem_addr=newsk;
			newsk->mem_len=len;
			kfree_skb(newsk, FREE_WRITE);
			continue;
		}

		IS_SKB(orig);
		IS_SKB(newsk);
		memcpy(newsk,orig,len);
		newsk->list=NULL;
		newsk->magic=0;
		newsk->next=NULL;
		newsk->prev=NULL;
		newsk->mem_addr=newsk;
		newsk->h.raw+=((char *)newsk-(char *)orig);
		newsk->link3=NULL;
		newsk->sk=NULL;
		newsk->free=1;
	}
	while(0);

	restore_flags(flags);
	return(newsk);
}

```

* There are a lot of `cli();` in this file. It is used to ensure the interupt flag is cleaned for some funcions will use `sti()` inside.

* `do{...}while(0)` nice trick. We can use `continue` inside and if the condition statisfied, it will go out from the loop. No need a `break`

* `newsk=alloc_skb(len,GFP_KERNEL);	/* May sleep */` when use alloc memory, we may swap out some memory page, this will cause a sleep because of writing to disk.

* volatile and pointer

![skbuff02.png](imgs/skbuff02.png)
![skbuff03.png](imgs/skbuff03.png)

So `struct sk_buff *volatile* list` means

`*list` is volatile, which is the the sk_buff pointer, but the pointer ot pointer `**list` is not.

------------

## net/inet/sock

```c
#define SOCK_ARRAY_SIZE	64

struct proto {
  struct sk_buff *	(*wmalloc)(struct sock *sk,
				    unsigned long size, int force,
				    int priority);
  ....

  struct sock *		sock_array[SOCK_ARRAY_SIZE];
  char			name[80];
};

```

* `struct sock *		sock_array[SOCK_ARRAY_SIZE];` this is a hash table to record which ports are used. The key is port number.

```c
static int
sk_inuse(struct proto *prot, int num)
{
  struct sock *sk;

  for(sk = prot->sock_array[num & (SOCK_ARRAY_SIZE -1 )];
      sk != NULL;
      sk=sk->next) {
	if (sk->num == num) return(1);
  }
  return(0);
}
```

This function is to check the port usage. Classical resolving hash conflict method.

-------------

```c
#define PROT_SOCK	1024	/* Sockets 0-1023 can't be bound too unless you are superuser */


unsigned short
get_new_socknum(struct proto *prot, unsigned short base)
{
  static int start=0;

  /*
   * Used to cycle through the port numbers so the
   * chances of a confused connection drop.
   */
  int i, j;
  int best = 0;
  int size = 32767; /* a big num. */
  struct sock *sk;

  if (base == 0) base = PROT_SOCK+1+(start % 1024);
  if (base <= PROT_SOCK) {
	base += PROT_SOCK+(start % 1024);
  }

  /* Now look through the entire array and try to find an empty ptr. */
  for(i=0; i < SOCK_ARRAY_SIZE; i++) {
	j = 0;
	sk = prot->sock_array[(i+base+1) &(SOCK_ARRAY_SIZE -1)];
	while(sk != NULL) {
		sk = sk->next;
		j++;
	}
	if (j == 0) {
		start =(i+1+start )%1024;
		DPRINTF((DBG_INET, "get_new_socknum returning %d, start = %d\n",
							i + base + 1, start));
		return(i+base+1);
	}

  /* xitongsys
  Find the shortest linked-list port
  */
	if (j < size) {
		best = i;
		size = j;
	}
  }

  /* Now make sure the one we want is not in use. */
  while(sk_inuse(prot, base +best+1)) {
	best += SOCK_ARRAY_SIZE;
  }
  DPRINTF((DBG_INET, "get_new_socknum returning %d, start = %d\n",
						best + base + 1, start));
  return(best+base+1);
}
```

* protocol has an array of sock `sock_array` with `PORT_SOCK` size. This array is a hash table for recording the ports used for each protocol. Using a linked-list method to resolve the hast conflict. 

------------

```c
void
put_sock(unsigned short num, struct sock *sk)
{
  struct sock *sk1;
  struct sock *sk2;
  int mask;

  DPRINTF((DBG_INET, "put_sock(num = %d, sk = %X\n", num, sk));
  sk->num = num;
  sk->next = NULL;
  num = num &(SOCK_ARRAY_SIZE -1);

  /* We can't have an interupt re-enter here. */
  cli();
  if (sk->prot->sock_array[num] == NULL) {
	sk->prot->sock_array[num] = sk;
	sti();
	return;
  }
  sti();

  /* xitongsys
    addr is in big endian order. So this mask is not the subset mask(actually is reversed)
  */
  for(mask = 0xff000000; mask != 0xffffffff; mask = (mask >> 8) | mask) {
	if ((mask & sk->saddr) &&
	    (mask & sk->saddr) != (mask & 0xffffffff)) {
		mask = mask << 8;
		break;
	}
  }
  DPRINTF((DBG_INET, "mask = %X\n", mask));

  cli();
  sk1 = sk->prot->sock_array[num];

  /* xitongsys
  the list is sorted by the addr
  */
  for(sk2 = sk1; sk2 != NULL; sk2=sk2->next) {
	if (!(sk2->saddr & mask)) {
		if (sk2 == sk1) {
			sk->next = sk->prot->sock_array[num];
			sk->prot->sock_array[num] = sk;
			sti();
			return;
		}
		sk->next = sk2;
		sk1->next= sk;
		sti();
		return;
	}
	sk1 = sk2;
  }

  /* Goes at the end. */
  sk->next = NULL;
  sk1->next = sk;
  sti();
}
```

--------------

```c
static int
inet_listen(struct socket *sock, int backlog)
{
  struct sock *sk;

  sk = (struct sock *) sock->data;
  if (sk == NULL) {
	printk("Warning: sock->data = NULL: %d\n" ,__LINE__);
	return(0);
  }

  /* We may need to bind the socket. */
  if (sk->num == 0) {
	sk->num = get_new_socknum(sk->prot, 0);
	if (sk->num == 0) return(-EAGAIN);
	put_sock(sk->num, sk);
	sk->dummy_th.source = ntohs(sk->num);
  }

  /* We might as well re use these. */ 
  sk->max_ack_backlog = backlog;
  if (sk->state != TCP_LISTEN) {
	sk->ack_backlog = 0;
	sk->state = TCP_LISTEN;
  }
  return(0);
}

```

* `max_ack_backlog` max number of accept queue

---------------

```c
/* The peer socket should always be NULL. */
static int
inet_release(struct socket *sock, struct socket *peer)
{
  struct sock *sk;

  sk = (struct sock *) sock->data;
  if (sk == NULL) return(0);

  DPRINTF((DBG_INET, "inet_release(sock = %X, peer = %X)\n", sock, peer));
  sk->state_change(sk);

  /* Start closing the connection.  This may take a while. */
  /*
   * If linger is set, we don't return until the close
   * is complete.  Other wise we return immediately. The
   * actually closing is done the same either way.
   */
  if (sk->linger == 0) {
	sk->prot->close(sk,0);
	sk->dead = 1;
  } else {
	DPRINTF((DBG_INET, "sk->linger set.\n"));
	sk->prot->close(sk, 0);
	cli();

    /* xitongsys
    lingertime to wait
    */
	if (sk->lingertime)
		current->timeout = jiffies + HZ*sk->lingertime;
	while(sk->state != TCP_CLOSE && current->timeout>0) {
		interruptible_sleep_on(sk->sleep);
		if (current->signal & ~current->blocked) {
			break;
#if 0
			/* not working now - closes can't be restarted */
			sti();
			current->timeout=0;
			return(-ERESTARTSYS);
#endif
		}
	}
	current->timeout=0;
	sti();
	sk->dead = 1;
  }
  sk->inuse = 1;

  /* This will destroy it. */
  release_sock(sk);
  sock->data = NULL;
  DPRINTF((DBG_INET, "inet_release returning\n"));
  return(0);
}

```

* TIME_WAIT and linger

![linger01.png](imgs/linger01.png)
![linger02.png](imgs/linger02.png)
![linger03.png](imgs/linger03.png)

-------

## net/inet/dev

```c
int
ip_addr_match(unsigned long me, unsigned long him)
{
  int i;
  unsigned long mask=0xFFFFFFFF;
  DPRINTF((DBG_DEV, "ip_addr_match(%s, ", in_ntoa(me)));
  DPRINTF((DBG_DEV, "%s)\n", in_ntoa(him)));

  if (me == him) 
  	return(1);
  for (i = 0; i < 4; i++, me >>= 8, him >>= 8, mask >>= 8) {
	if ((me & 0xFF) != (him & 0xFF)) {
		/*
		 * The only way this could be a match is for
		 * the rest of addr1 to be 0 or 255.
		 */
		if (me != 0 && me != mask) return(0);
		return(1);
	}
  }
  return(1);
}
```

* Check `me` and `him` match. `me` can be a broadcast address 0/255

-----------

```c
/*
 * The old interface to fetch a packet from a device driver.
 * This function is the base level entry point for all drivers that
 * want to send a packet to the upper (protocol) levels.  It takes
 * care of de-multiplexing the packet to the various modules based
 * on their protocol ID.
 *
 * Return values:	1 <- exit I can't do any more
 *			0 <- feed me more (i.e. "done", "OK"). 
 */
int
dev_rint(unsigned char *buff, long len, int flags, struct device *dev)
{
  static int dropping = 0;
  struct sk_buff *skb = NULL;
  unsigned char *to;
  int amount, left;
  int len2;

  if (dev == NULL || buff == NULL || len <= 0) return(1);
  if (flags & IN_SKBUFF) {
	skb = (struct sk_buff *) buff;
  } else {
	if (dropping) {
	  if (backlog != NULL)
	      return(1);
	  printk("INET: dev_rint: no longer dropping packets.\n");
	  dropping = 0;
	}

	skb = alloc_skb(sizeof(*skb) + len, GFP_ATOMIC);
	if (skb == NULL) {
		printk("dev_rint: packet dropped on %s (no memory) !\n",
		       dev->name);
		dropping = 1;
		return(1);
	}
	skb->mem_len = sizeof(*skb) + len;
	skb->mem_addr = (struct sk_buff *) skb;

	/* First we copy the packet into a buffer, and save it for later. */
	to = skb->data;
	left = len;
	len2 = len;
	while (len2 > 0) {
		amount = min(len2, (unsigned long) dev->rmem_end -
						(unsigned long) buff);
		memcpy(to, buff, amount);
		len2 -= amount;
		left -= amount;
		buff += amount;
		to += amount;

		/* xitongsys
		the dev data buffer(rmem) used a cyclic buffer and len is the total size to read. So if we touch the rmem_end, we should begin from the rmem_start.
		*/
		if ((unsigned long) buff == dev->rmem_end)
			buff = (unsigned char *) dev->rmem_start;
	}
  }
  skb->len = len;
  skb->dev = dev;
  skb->free = 1;

  netif_rx(skb);
  /* OK, all done. */
  return(0);
}
```

![dev01](imgs/dev01.png)
![dev02](imgs/dev02.png)

[ref](https://www.oreilly.com/library/view/linux-device-drivers/0596000081/ch14s03.html)

-------------

```c
/*
 * Receive a packet from a device driver and queue it for the upper
 * (protocol) levels.  It always succeeds.
 */
void
netif_rx(struct sk_buff *skb)
{
  /* Set any necessary flags. */
  skb->sk = NULL;
  skb->free = 1;
  
  /* and add it to the "backlog" queue. */
  IS_SKB(skb);
  skb_queue_tail(&backlog,skb);
   
  /* If any packet arrived, mark it for processing. */
  if (backlog != NULL) mark_bh(INET_BH);

  return;
}
```

* new packet to `backlog` queue

------

```c
/* Send (or queue for sending) a packet. */
void
dev_queue_xmit(struct sk_buff *skb, struct device *dev, int pri)
{
  int where = 0;		/* used to say if the packet should go	*/
				/* at the front or the back of the	*/
				/* queue.				*/

  DPRINTF((DBG_DEV, "dev_queue_xmit(skb=%X, dev=%X, pri = %d)\n",
							skb, dev, pri));

  if (dev == NULL) {
	printk("dev.c: dev_queue_xmit: dev = NULL\n");
	return;
  }
 
  IS_SKB(skb);
    
  skb->dev = dev;
  if (skb->next != NULL) {
	/* Make sure we haven't missed an interrupt. */
	dev->hard_start_xmit(NULL, dev);
	return;
  }

  if (pri < 0) {
	pri = -pri-1;
	where = 1;
  }

  if (pri >= DEV_NUMBUFFS) {
	printk("bad priority in dev_queue_xmit.\n");
	pri = 1;
  }

  if (dev->hard_start_xmit(skb, dev) == 0) {
	return;
  }

  /* Put skb into a bidirectional circular linked list. */
  DPRINTF((DBG_DEV, "dev_queue_xmit dev->buffs[%d]=%X\n",
					pri, dev->buffs[pri]));

  /* Interrupts should already be cleared by hard_start_xmit. */
  cli();
  skb->magic = DEV_QUEUE_MAGIC;
  if(where)
  	skb_queue_head(&dev->buffs[pri],skb);
  else
  	skb_queue_tail(&dev->buffs[pri],skb);
  skb->magic = DEV_QUEUE_MAGIC;
  sti();
}

```

* `hard_start_xmit` is the real send function implemented in ther drivers

* `dev->buffs` is priority indexed and it priority < 0 means put is at the head of `skb_queue`

----------------

## net/inet/ip

```c
/* IP flags. */
#define IP_CE		0x8000		/* Flag: "Congestion"		*/
#define IP_DF		0x4000		/* Flag: "Don't Fragment"	*/
#define IP_MF		0x2000		/* Flag: "More Fragments"	*/
#define IP_OFFSET	0x1FFF		/* "Fragment Offset" part	*/
```

![ip01](imgs/ip01.png)
![ip02](imgs/ip02.png)
![ip03](imgs/ip03.png)

* the offset unit is 8 Bytes

------------

```c
struct iphdr {
  unsigned char		ihl:4,
			version:4;
  unsigned char		tos;
  unsigned short	tot_len;
  unsigned short	id;
  unsigned short	frag_off;
  unsigned char		ttl;
  unsigned char		protocol;
  unsigned short	check;
  unsigned long		saddr;
  unsigned long		daddr;
  /*The options start here. */
};
```

* IPv4 header

![ip04](imgs/ip04.png)

-----------------

```c
uint32_t htonl(uint32_t hostlong);
uint16_t htons(uint16_t hostshort);
uint32_t ntohl(uint32_t netlong);
uint16_t ntohs(uint16_t netshort);

Description
The htonl() function converts the unsigned integer hostlong from host byte order to network byte order.
The htons() function converts the unsigned short integer hostshort from host byte order to network byte order.

The ntohl() function converts the unsigned integer netlong from network byte order to host byte order.

The ntohs() function converts the unsigned short integer netshort from network byte order to host byte order.

On the i386 the host byte order is Least Significant Byte first, whereas the network byte order, as used on the Internet, is Most Significant Byte first.
```

-----------------

```c
/*
 * This routine is called when an device driver (i.e. an
 * interface) is * ready to transmit a packet.
 */
 
void dev_tint(struct device *dev)
{
	int i;
	struct sk_buff *skb;
	
	for(i = 0;i < DEV_NUMBUFFS; i++) {
		while((skb=skb_dequeue(&dev->buffs[i]))!=NULL)
		{
			skb->magic = 0;
			skb->next = NULL;
			skb->prev = NULL;
			dev->queue_xmit(skb,dev,-i - 1);
			if (dev->tbusy)
				return;
		}
	}
}
```

* send data. `buffs` is hash indexed by packet priority.

------

```c
/*
 * This function gets called periodically, to see if we can
 * process any data that came in from some interface.
 *
 */
void
inet_bh(void *tmp)
{
  struct sk_buff *skb;
  struct packet_type *ptype;
  unsigned short type;
  unsigned char flag = 0;
  int nitcount;

  /* Atomically check and mark our BUSY state. */
  if (set_bit(1, (void*)&in_bh))
      return;

  /* Can we send anything now? */
  dev_transmit();
  
  /* Any data left to process? */
  while((skb=skb_dequeue(&backlog))!=NULL)
  {
  	nitcount=dev_nit;
	flag=0;
	sti();

    /* xitongsys
    trim the MAC header which has length of dev->hard_header_len
    */

       /*
	* Bump the pointer to the next structure.
	* This assumes that the basic 'skb' pointer points to
	* the MAC header, if any (as indicated by its "length"
	* field).  Take care now!
	*/
       skb->h.raw = skb->data + skb->dev->hard_header_len;
       skb->len -= skb->dev->hard_header_len;

       /*
	* Fetch the packet protocol ID.  This is also quite ugly, as
	* it depends on the protocol driver (the interface itself) to
	* know what the type is, or where to get it from.  The Ethernet
	* interfaces fetch the ID from the two bytes in the Ethernet MAC
	* header (the h_proto field in struct ethhdr), but drivers like
	* SLIP and PLIP have no alternative but to force the type to be
	* IP or something like that.  Sigh- FvK
	*/
       type = skb->dev->type_trans(skb, skb->dev);

	/*
	 * We got a packet ID.  Now loop over the "known protocols"
	 * table (which is actually a linked list, but this will
	 * change soon if I get my way- FvK), and forward the packet
	 * to anyone who wants it.
	 */
	for (ptype = ptype_base; ptype != NULL; ptype = ptype->next) {
		if (ptype->type == type || ptype->type == NET16(ETH_P_ALL)) {
			struct sk_buff *skb2;

            /* xitongsys
            nitcount is number of devs which needs this packet. So we should copy the skb.
            */

			if (ptype->type==NET16(ETH_P_ALL))
				nitcount--;
			if (ptype->copy || nitcount) {	/* copy if we need to	*/
				skb2 = alloc_skb(skb->mem_len, GFP_ATOMIC);
				if (skb2 == NULL) 
					continue;
				memcpy(skb2, (const void *) skb, skb->mem_len);
				skb2->mem_addr = skb2;
				skb2->h.raw = (unsigned char *)(
				    (unsigned long) skb2 +
				    (unsigned long) skb->h.raw -
				    (unsigned long) skb
				);
				skb2->free = 1;
			} else {
				skb2 = skb;
			}

			/* This used to be in the 'else' part, but then
			 * we don't have this flag set when we get a
			 * protocol that *does* require copying... -FvK
			 */
			flag = 1;

			/* Kick the protocol handler. */
			ptype->func(skb2, skb->dev, ptype);
		}
	}

	/*
	 * That's odd.  We got an unknown packet.  Who's using
	 * stuff like Novell or Amoeba on this network??
	 */
	if (!flag) {
		DPRINTF((DBG_DEV,
			"INET: unknown packet type 0x%04X (ignored)\n", type));
		skb->sk = NULL;
		kfree_skb(skb, FREE_WRITE);
	}

	/* Again, see if we can transmit anything now. */
	dev_transmit();
	cli();
  }
  in_bh = 0;
  sti();
  dev_transmit();
}


```

* `nit_count = dev_nit` the number of capturing all eth packets dev(something like TAP dev). So in `dev_add_pack`, 

```c
  if(pt->type==NET16(ETH_P_ALL))
  	dev_nit++;	/* I'd like a /dev/nit too one day 8) */
```

the dev type is ETH_P_ALL.

-------------

```c
/* Add a protocol ID to the list.  This will change soon. */
void
dev_add_pack(struct packet_type *pt)
{
  struct packet_type *p1;
  pt->next = ptype_base;

  /* Don't use copy counts on ETH_P_ALL. Instead keep a global
     count of number of these and use it and pt->copy to decide
     copies */
  pt->copy=0;
  if(pt->type==NET16(ETH_P_ALL))
  	dev_nit++;	/* I'd like a /dev/nit too one day 8) */
  else
  {
  	/* See if we need to copy it. */
  	for (p1 = ptype_base; p1 != NULL; p1 = p1->next) {
		if (p1->type == pt->type) {
			pt->copy = 1;
			break;
		}
	  }
  }
```

* this is a `packet_type` list which record all different eth types. If there are multiple same types on the list, set the `copy` flag.

-----------

```





  handler[int(*func) (struct sk_buff *, struct device *,
				 struct packet_type *);]
  handler used to transport the packet to upper layer or process it.

    ^
    |
    |
    |
    |                   ptype_base: packet_type list (网络层)
ptype1(ETH_P_IP) -> ptype2(ETH_P_ARP) -> ptype3(ETH_P_ALL)

    ^
    |
    |   void inet_bh(void *tmp) this function gets called 
    |   periodically and transport packet from backlog to
    |   packet_type list.
    |
-----------------------------------------------------------------------------
backlog: skb list, dev received packetd append to this list
    ^
    |
    |  dev receives packets and append to backlog list 
    |
    |
dev_base: dev list (链路层data link)

            dev1 -> dev2 -> dev3
              |
              v
            skb buffs[priority] ( this is the send buffers hash table indexed by priority of packet)
```

---------------------

## net/inet/arp

![arp01](imgs/arp01.png)

* include/linux/if_arp.h

```c
/*
 * Address Resolution Protocol.
 *
 * See RFC 826 for protocol description.  ARP packets are variable
 * in size; the arphdr structure defines the fixed-length portion.
 * Protocol type values are the same as those for 10 Mb/s Ethernet.
 * It is followed by the variable-sized fields ar_sha, arp_spa,
 * arp_tha and arp_tpa in that order, according to the lengths
 * specified.  Field names used correspond to RFC 826.
 */
struct arphdr {
  unsigned short	ar_hrd;		/* format of hardware address	*/
  unsigned short	ar_pro;		/* format of protocol address	*/
  unsigned char		ar_hln;		/* length of hardware address	*/
  unsigned char		ar_pln;		/* length of protocol address	*/
  unsigned short	ar_op;		/* ARP opcode (command)		*/

  /* The rest is variable in size, according to the sizes above. */
#if 0
  unsigned char		ar_sha[];	/* sender hardware address	*/
  unsigned char		ar_spa[];	/* sender protocol address	*/
  unsigned char		ar_tha[];	/* target hardware address	*/
  unsigned char		ar_tpa[];	/* target protocol address	*/
#endif	/* not actually included! */
};

```

------------

```c
/* This will find an entry in the ARP table by looking at the IP address. */
static struct arp_table *
arp_lookup(unsigned long paddr)
{
  struct arp_table *apt;
  unsigned long hash;

  DPRINTF((DBG_ARP, "ARP: lookup(%s)\n", in_ntoa(paddr)));

  /* We don't want to ARP ourselves. */
  if (chk_addr(paddr) == IS_MYADDR) {
	printk("ARP: ARPing my own IP address %s !\n", in_ntoa(paddr));
	return(NULL);
  }

  /* Loop through the table for the desired address. */
  hash = htonl(paddr) & (ARP_TABLE_SIZE - 1);
  cli();
  apt = arp_tables[hash];
  while(apt != NULL) {
	if (apt->ip == paddr) {
		sti();
		return(apt);
	}
	apt = apt->next;
  }
  sti();
  return(NULL);
}
```

* `paddr` protocol address, here is IP. This function find the MAC address corresponding to the IP from the `arp_tables`

* `arp_tables` is a classic hash table

----------

```c
static struct timer_list arp_timer;

static void arp_queue_ticker(unsigned long data);

static void arp_queue_kick(void)
{
	arp_timer.expires = 500;	/* 5 seconds */
	arp_timer.data = 0;
	arp_timer.function = arp_queue_ticker;
	del_timer(&arp_timer);
	add_timer(&arp_timer);
}

static void arp_queue_ticker(unsigned long data/*UNUSED*/)
{
	arp_send_q();
	if (skb_peek(&arp_q))
		arp_queue_kick();
}

```

* create a timer(5seconds) to send packet in arp queue `arp_send_q()`

--------------------

```c
  /*
   * Since we updated the ARP cache, we might have enough
   * information to send out some previously queued IP
   * datagrams....
   */
  arp_send_q();
```

* `arp_q` is a queue for some IP packet which not known the MAC address


```c
/* Queue an IP packet, while waiting for the ARP reply packet. */
void
arp_queue(struct sk_buff *skb)
{
  cli();
  skb->tries = ARP_MAX_TRIES;

  if (skb->next != NULL) {
	sti();
	printk("ARP: arp_queue skb already on queue magic=%X.\n", skb->magic);
	return;
  }
  if(arp_q==NULL)
  	arp_queue_kick();
  skb_queue_tail(&arp_q,skb);
  skb->magic = ARP_QUEUE_MAGIC;
  sti();
}
```

* This function append new packet to `arp_q` and is called by drivers, something like

```c
    /* Fill in the ethernet header. */
    if (!skb->arp  &&  dev->rebuild_header(skb->data, dev)) {
		skb->dev = dev;
		arp_queue (skb);
		return 0;
    }
    skb->arp=1;
```

* `skb->arp == 1` means this packet has already filled ethernet header, which means it already has the MAC address. If not, it will push to the `arp_q`.

-------

```c
/* Create and send an ARP REQUEST packet. */
void
arp_send(unsigned long paddr, struct device *dev, unsigned long saddr)
{
```

* no tricky, just fill the arp request packet and send to dev

-------------

```c
/* Create an ARP entry.  The caller should check for duplicates! */
static struct arp_table *
arp_create(unsigned long paddr, unsigned char *addr, int hlen, int htype)
{
  struct arp_table *apt;
  unsigned long hash;

  DPRINTF((DBG_ARP, "ARP: create(%s, ", in_ntoa(paddr)));
  DPRINTF((DBG_ARP, "%s, ", eth_print(addr)));
  DPRINTF((DBG_ARP, "%d, %d)\n", hlen, htype));

  apt = (struct arp_table *) kmalloc(sizeof(struct arp_table), GFP_ATOMIC);
  if (apt == NULL) {
	printk("ARP: no memory available for new ARP entry!\n");
	return(NULL);
  }

  /* Fill in the allocated ARP cache entry. */
  hash = htonl(paddr) & (ARP_TABLE_SIZE - 1);
  apt->ip = paddr;
  apt->hlen = hlen;
  apt->htype = htype;
  apt->flags = (ATF_INUSE | ATF_COM);		/* USED and COMPLETED entry */
  memcpy(apt->ha, addr, hlen);
  apt->last_used = jiffies;
  cli();
  apt->next = arp_tables[hash];
  arp_tables[hash] = apt;
  sti();
  return(apt);
}

```

* create a arp table entry in `arp_tables`

-------------

## net/inet/route

```c
/* This is an entry in the IP routing table. */
struct rtable {
  struct rtable		*rt_next;
  unsigned long		rt_dst;
  unsigned long		rt_mask;
  unsigned long		rt_gateway;
  unsigned char		rt_flags;
  unsigned char		rt_metric;
  short			rt_refcnt;
  unsigned long		rt_use;
  unsigned short	rt_mss, rt_mtu;
  struct device		*rt_dev;
};

```

----------------

* include/linux/route.h

```c
#define	RTF_UP		0x0001		/* route useable		*/
#define	RTF_GATEWAY	0x0002		/* destination is a gateway	*/
#define	RTF_HOST	0x0004		/* host entry (net otherwise)	*/
#define RTF_REINSTATE	0x0008		/* re-instate route after tmout	*/
#define	RTF_DYNAMIC	0x0010		/* created dyn. (by redirect)	*/
#define	RTF_MODIFIED	0x0020		/* modified dyn. (by redirect)	*/
```

------------------